## NIM Agent Blueprint for Generative Virtual Screening in Drug Discovery
Drug discovery is a complex and resource intensive process that traditionally involves screening large libraries of molecules to identify potential candidates for therapeutic development. Virtual screening workflows powered by generative AI offer a transformative approach to this process, enabling researchers to efficiently explore vast chemical and biological spaces, predict molecular interactions, and design optimized drug candidates.

This example notebook demonstrates how to connect BioNeMo NIMs (NVIDIA Inference Microservices) to perform a few key steps of a virtual screening workflow. Importantly, each step in the workflow is powered by NVIDIA GPUs and highly performant AI models in each category: MSA-Search (MMSeqs2) for sequence alignment generation, OpenFold2 for folding, GenMol for molecular generation, and DiffDock for protein-ligand docking.

Below, we illustrate this workflow using an example protein and molecule of interest, the SARS-CoV-2 main protease and the antiviral compound Nirmatrelvir. However, users are free to define any protein and molecule of their choosing as long as they meet the input requirements of the NIMs.

All of these capabilities are enabled by NVIDIA NIM and NVIDIA NIM Blueprints. For more details, please visit [NVIDIA NIM Blueprints](https://build.nvidia.com/nim/blueprints).

### BioNeMo NIMs Configuration
Before proceeding, ensure you have completed the [Helm Deployment Prerequisites](../generative-virtual-screening-chart/README.md) including setting up a secret named `ngc-registry-secret` with your personal run key for your NGC Organization. This step is required before installing the Helm chart.

### Setting Up Your Environment
First, we will set up our environment to make sure we can invoke the deployed NIMs.
We use the Python `requests` library make APIs call to the NIMs. The generated molecules and folded proteins are visualized using `RDKit` and `py3Dmol`. `RDKit` is a robust toolkit for cheminformatics, enabling the visualization and manipulation of molecular structures in 2D. `py3Dmol`, provides interactive 3D visualization of molecules, making it ideal for exploring protein-ligand interactions in this notebook.

In [ ]:
! pip install py3Dmol
! pip install rdkit

import os

import py3Dmol
import requests
from IPython.display import display
from rdkit import Chem
from rdkit.Chem import Draw

Next, we define variables for the IP addresses and ports of each locally-deployed NIM To interact with. Set the following environment variables corresponding to the ports you forwarded during the [Helm Deployment Prerequisites](../generative-virtual-screening-chart/README.md#configure-cluster--helm-deployment)

In [ ]:

MSA_HOST = 'http://localhost:8081'
OPENFOLD2_HOST = 'http://localhost:8082'
GENMOL_HOST = 'http://localhost:8083'
DIFFDOCK_HOST = 'http://localhost:8084'

Before sending requests it is best practice to check the NIMs are in a live and ready state. This can be done using the `/v1/health/ready` endpoint of each NIM:

In [ ]:
for nim_url in [MSA_HOST, OPENFOLD2_HOST, GENMOL_HOST, DIFFDOCK_HOST]:
    try:
        response = requests.get(os.path.join(nim_url, "v1/health/ready"))
        if response.status_code == 200:
            print("Response:", response.text)
        else:
            print(f"Failed to invoke {nim_url} with status code: {response.status_code}")
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")

### Protein Folding with MMSeqs2 and OpenFold2
Once a target protein of interest has been identified, the first step of this virtual screening demonstration is to generate a plausible structure of that protein sequence. We do so by using OpenFold2, a generative, transformer-based model that translates protein sequences into protein structures.

To help predicting spatial constraints between amino acid residues and improving the accuracy of predicted protein structures, OpenFold2 leverages the MSA-search NIM (powered by MMSeqs2 - a GPU accelerated toolkit for protein database search and Multiple Sequence Alignment) as a key input to provide co-evolutionary information and infer structural features.

To demonstrate this part of our workflow, we begin with an example protein sequence. Here, we choose the SARS-CoV-2 main protease as our starting sequence. We query the MSA-search endpoint with this sequence and the model returns the alignment data according to each of the sequence search supported databases.

Please note that this step may take a few minutes to be finished (depending on the GPU type), which is much slower than the other two inference steps that usually requires less than a minute

In [ ]:
# SARS CoV-2 main protease sequence
protein = "SGFRKMAFPSGKVEGCMVQVTCGTTTLNGLWLDDVVYCPRHVICTSEDMLNPNYEDLLIRKSNHNFLVQAGNVQLRVIGHSMQNCVLKLKVDTANPKTPKYKFVRIQPGQTFSVLACYNGSPSGVYQCAMRPNFTIKGSFLNGSCGSVGFNIDYDCVSFCYMHHMELPTGVHAGTDLEGNFYGPFVDRQTAQAAGTDTTITVNVLAWLYAAVINGDRWFLNRFTTTLNDFNLVAMKYNYEPLTQDHVDILGPLSAQTGIAVLDMCASLKELLQNGMNGRTILGSALLEDEFTPFDVVRQCSGVTFQ"

In [ ]:
# Query MSA-search with sequence above
msa_response = requests.post(
    f'{MSA_HOST}/biology/colabfold/msa-search/predict',
    json={
        'sequence': protein,
        'e_value': 0.0001,
        'iterations': 1,    
        'search_type': 'alphafold2',
        'output_alignment_formats' : ['fasta', 'a3m'],
        'databases': ['Uniref30_2302', 'colabfold_envdb_202108', 'PDB70_220313']
    }).json()

We pass the original sequence and the alignments created by each of the selected databases of the sequence to OpenFold2 to predict protein structure.

In [ ]:
print(msa_response)

In [ ]:
# Query OpenFold2 
of2_response = requests.post(
    f'{OPENFOLD2_HOST}/biology/openfold/openfold2/predict-structure-from-msa-and-template',
    json={
        'sequence': protein,
        'use_templates': False,
        'relaxed_prediction': False,
        'alignments': msa_response['alignments'],
    }).json()
# Receive protein structure for SARS CoV-2 protease
folded_protein = of2_response["structures_in_ranked_order"].pop(0)["structure"]

Inspect the folded protein pdb content

In [ ]:
print(folded_protein[:648])

Let us use py3Dmol to visualize the folded protein generated by OpenFold2

In [ ]:
view = py3Dmol.view(width=800, height=600)
view.addModel(folded_protein, "pdb")
view.setStyle({'model': -1}, {"cartoon": {'color': 'spectrum'}})
view.zoomTo()
view.show()

### Molecular Generation with GenMol
The next step in our workflow is generating molecules with optimized chemical properties starting from a seed molecule of interest. Here, molecular generation is powered by GenMol, a masked diffusion model trained on Sequential Attachment based Fragment Embedding (SAFE) representations aimed at generating and optimizing molecules according to user-defined objectives.

Here, we begin with a fragmented formatted Nirmatrelvir, an active component of the Covid treatment Paxlovid, aimed at targeting the SARS-CoV-2 main protease. By using this molecule as the input to GenMol, the model will return 5 generated molecules with the highest chemical similarity to the seed molecule, Nirmatrelvir. The user is able to specify the number of generated molecules to return when querying the GenMol NIM.

Additionally, the user is able to specify chemical properties to optimize for. In this example, we have chosen to optimize the Quantitative Estimate of Drug-Likeness (QED) score, to produce molecules with favorable pharmacokinetic properties.

To Further experiment with GenMol please refer to our [GiHub Examples](https://github.com/NVIDIA/bionemo-examples/tree/main/examples/nims/genmol#Setup)


In [ ]:
# Nirmatrelvir
molecule = "C12OC3C(O)C1O.C3O.[*{25-25}]"

In [ ]:
# Query GenMol
genmol_response = requests.post(
    f'{GENMOL_HOST}/generate',
    json={
        'smiles': molecule,
        'num_molecules': 5,
        "temperature":1,
        "noise":0.2,
        "step_size":4,
        "scoring":"QED"
    }).json()
# Prepare GenMol output for DiffDock
generated_ligands = '\n'.join([ v['smiles']for v in genmol_response['molecules']])

In [ ]:
# Print the smiles and score of each generated molecule
print("\n".join([f"SMILES: {mol['smiles']}, Score: {mol['score']}" for mol in genmol_response['molecules']]))

### Protein-Ligand Docking with DiffDock

After obtaining the molecules with optimized QED scores, we can predict their binding poses to the receptor target. Here, we apply DiffDock, a state-of-the-art generative model that predicts the 3D structure of a protein-ligand complex, to find out the best (most probable) binding poses. A highlighted feature from DiffDock is that a presumed binding pocket, which usually can be characterized only from experimental 3D structures, is not needed (a.k.a., blind-docking). This feature is very useful for AI folded protein structures, as it is able to locate all regions on the protein surface to be bound by drug molecules, providing insights for further downstream investigations.

The optimized DiffDock also provides the batch-docking function, by which we can concatenate multiple molecules into one request of docking, each of them will be also sampled for multiple poses (i.e., num_poses=10 in this example). In the output, the predicted docking poses for each molecule is sorted by a confidence score that inferenced from a confidence model.

In [ ]:
diffdock_response = requests.post(
    f'{DIFFDOCK_HOST}/molecular-docking/diffdock/generate',
    json={
        'protein': folded_protein,
        'ligand': generated_ligands,
        'ligand_file_type': 'txt',
        'num_poses': 10,
        'time_divisions': 20,
        'num_steps': 18,
    }).json()

In this step, we construct a molecule from each SMILES string that was generated by GenMol:

In [ ]:

ligands = diffdock_response["ligand"].split('\n')

mols = [
    Chem.MolFromSmiles(smiles)
    for smiles, status in zip(ligands, diffdock_response["status"])
    if status == "success" and Chem.MolFromSmiles(smiles)]

legends = [smiles 
           for smiles, status in zip(ligands, diffdock_response["status"]) 
           if status == "success"]

img = Draw.MolsToGridImage(mols, legends=legends, subImgSize=(300, 300))
display(img)


The following code block visualizes the pose with the highest confidence for each generated molecule

In [ ]:

def visualize_3d_ligand(mol_block):
    mol_3d = Chem.MolFromMolBlock(mol_block, sanitize=False)
    if mol_3d is None:
        print("Could not parse MolBlock.")
        return
    pdb_block = Chem.MolToPDBBlock(mol_3d)

    viewer = py3Dmol.view(width=800, height=400)
    viewer.addModel(pdb_block, "pdb")
    viewer.setStyle({'stick': {}})
    viewer.zoomTo()
    return viewer.show()

for i in range(len(diffdock_response['ligand_positions'])):
    print(f"Visualizing top ligand for {ligands[i]}")
    visualize_3d_ligand(diffdock_response['ligand_positions'][i][0])

## End-to-End Virtual Screening with NIMs

In this Blueprint, we illustrate the ability of BioNeMo NIMs to work in concert to generate meaningful predictions in a small virtual screening workflow. We hope this underscores to the user how easy the tools are to query and assimilate, and how flexible a workflow of this sort can be.

Blueprints are provided open-source and designed to be extended to fit your advanced use case. For example, you might consider:
- Utilizing a custom MSA database with the MSA Search NIM to tailor the results to your use case.
- Using your own custom structural template database with the OpenFold2 NIM to inform structural prediction.
- Adding advanced oracles for molecule generation with GenMol to generate candidates with specific properties.

## Build with this Blueprint

Now that you have successfully run the Generative Virtual Screening Blueprint, consider the following steps:
- Learn more about [NVIDIA AI Enterprise](https://docs.nvidia.com/ai-enterprise/index.html) and how to get advanced support with NIMs.
- Try the Generative Virtual Screening Blueprint demonstration on [build.nvidia.com](https://build.nvidia.com/nvidia/generative-virtual-screening-for-drug-discovery) to see further examples of the API usage and visualization.
